#Installing Dependencies

In [ ]:
!mkdir data
!mkdir data/clariq
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/dev.tsv
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/train.tsv
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/question_bank.tsv
!mv data/clariq/train.tsv data/clariq/train_original.tsv

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/clariq’: File exists
--2023-04-29 10:36:50--  https://github.com/aliannejadi/ClariQ/raw/master/data/dev.tsv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/dev.tsv [following]
--2023-04-29 10:36:50--  https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 585998 (572K) [text/plain]
Saving to: ‘dev.tsv.1’

dev.tsv.1           100%[===================>] 572.26K  --.-KB/s    in 0.01s   

2023-04-29 10:36:50 (38.0 MB/

In [ ]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Preparing Dataset

In [ ]:
import pandas as pd
data_path = "./data/"

train = pd.read_csv(data_path+"clariq/train_original.tsv", sep="\t")
valid = pd.read_csv(data_path+"clariq/dev.tsv", sep="\t")

train = train.groupby('topic_id', as_index=False).first()
train = train[['initial_request','clarification_need']]
train.columns = ["text", "label"]

valid = valid.groupby('topic_id', as_index=False).first()
valid = valid[['initial_request','clarification_need']]
valid.columns = ["text", "label"]

In [ ]:
train.head()

,text,label
0,Tell me about Obama family tree.,2
1,Find information on French Lick Resort and Cas...,3
2,How to get organised?,2
3,How to change the toilet in the house,3
4,Find information about Mitchell College.,4


In [ ]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(train)
dataset_val = Dataset.from_pandas(valid)

In [ ]:
dataset_train[100]

{'text': "I'm looking for information on duchess county tourism", 'label': 3}

#Training

In [ ]:
! pip install torch
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
MODEL = 'albert-base-v2'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets_train = dataset_train.map(tokenize_function, batched=True)
tokenized_datasets_val = dataset_val.map(tokenize_function, batched=True)


Map:   0%|          | 0/187 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=5)
model = model.to(device)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
# Load Precision,recall and F1 score
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Calculate accuracy using accuracy_score function
    accuracy = metric.compute(predictions=predictions, references=labels)
    # Calculate precision using precision_score function
    precision_score = precision.compute(predictions=predictions, references=labels,average='weighted')
    # Calculate recall using recall_score function
    recall_score = recall.compute(predictions=predictions, references=labels,average='weighted')
    # Calculate f1 using f1_score function
    f1_score = f1.compute(predictions=predictions, references=labels,average='weighted')
    return accuracy

In [ ]:
from transformers import TrainingArguments, Trainer

# Increase training rate
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",save_strategy="epoch",
                                    num_train_epochs=10,
                                    learning_rate=0.0001, per_device_train_batch_size=4, per_device_eval_batch_size=4,
                                    warmup_steps=500, weight_decay=0.01, logging_dir="./logs", logging_steps=10,
                                    load_best_model_at_end=True, metric_for_best_model="accuracy", greater_is_better=True)


In [ ]:
# CUDA_LAUNCH_BLOCKING=1
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.429800,1.295491,0.460000
2,1.308300,1.235257,0.420000
3,1.302900,1.206102,0.480000
4,1.368900,1.322215,0.440000
5,1.182200,1.411127,0.420000
6,1.310700,1.289469,0.420000
7,1.445800,1.252275,0.300000
8,1.311600,1.256733,0.420000
9,1.306000,1.241736,0.420000
10,1.337500,1.252978,0.420000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=470, training_loss=1.3426998990647336, metrics={'train_runtime': 218.4251, 'train_samples_per_second': 8.561, 'train_steps_per_second': 2.152, 'total_flos': 44702703175680.0, 'train_loss': 1.3426998990647336, 'epoch': 10.0})

In [ ]:
PATH = "/content/drive/MyDrive/"
%cd $PATH
%mkdir NLP
%cd NLP
# Save model to local directory
trainer.save_model("./"+MODEL)
%cd /content

/content/drive/MyDrive
mkdir: cannot create directory ‘NLP’: File exists
/content/drive/MyDrive/NLP
/content


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#Evaluation


In [ ]:
# Load dataset
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/test.tsv

--2023-04-29 11:09:53--  https://github.com/aliannejadi/ClariQ/raw/master/data/test.tsv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/test.tsv [following]
--2023-04-29 11:09:53--  https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2457 (2.4K) [text/plain]
Saving to: ‘test.tsv.2’

test.tsv.2          100%[===================>]   2.40K  --.-KB/s    in 0s      

2023-04-29 11:09:53 (46.5 MB/s) - ‘test.tsv.2’ saved [2457/2457]



In [ ]:
test = pd.read_csv(data_path+"clariq/test.tsv", sep="\t")
test = test.groupby('topic_id', as_index=False).first()
test.head()
test = test[['initial request']]
test.columns = ["text"]

dataset_test = Dataset.from_pandas(test)

tokenized_datasets_test = dataset_test.map(tokenize_function, batched=True)


Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
predicted = trainer.predict(tokenized_datasets_test)

In [ ]:
# Load dataset
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/test_with_labels.tsv
test_with_label = pd.read_csv(data_path+"clariq/test_with_labels.tsv", sep="\t")
test_with_label = test_with_label.groupby('topic_id', as_index=False).first()
test_with_label.head()
test_with_label = test_with_label[['initial_request','clarification_need']]
test_with_label.columns = ["text","labels"]

dataset_test_with_label = Dataset.from_pandas(test_with_label)

tokenized_datasets_test_with_label = dataset_test_with_label.map(tokenize_function, batched=True)



--2023-04-29 11:09:56--  https://github.com/aliannejadi/ClariQ/raw/master/data/test_with_labels.tsv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/test_with_labels.tsv [following]
--2023-04-29 11:09:56--  https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/test_with_labels.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1146708 (1.1M) [text/plain]
Saving to: ‘test_with_labels.tsv.2’

test_with_labels.ts 100%[===================>]   1.09M  --.-KB/s    in 0.02s   

2023-04-29 11:09:56 (62.1 MB/s) - ‘test_with_labels.tsv.2’ saved [1146708/1146708]



Map:   0%|          | 0/61 [00:00<?, ? examples/s]

In [ ]:
import torch
# Take 0th
pred = predicted[0]

# Take softmax
pred = torch.softmax(torch.from_numpy(pred),dim=1)
# pred = soft(pred)
pred = np.argmax(pred,axis=-1)
acc = metric.compute(predictions=pred, references=test_with_label['labels'])
precision = precision.compute(predictions=pred, references=test_with_label['labels'],average='weighted')
recall = recall.compute(predictions=pred, references=test_with_label['labels'],average='weighted')
f1 = f1.compute(predictions=pred, references=test_with_label['labels'],average='weighted')
print("Accuracy: ",acc)
print("Precision: ",precision)
print("Recall: ",recall)
print("F1: ",f1)


Accuracy:  {'accuracy': 0.6065573770491803}
Precision:  {'precision': 0.49672131147540977}
Recall:  {'recall': 0.6065573770491803}
F1:  {'f1': 0.5454678025009418}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
